<div class="alert alert-block alert-success">
    
# FIT5196 Task 1 in Assessment 1
#### Student Name: Deshui Yu      Liangjing Yang
#### Student ID: 34253599      34060871

Date: 21/08/2024


Environment: Python xxxx

Libraries used:
* re (for regular expression, installed and imported) 
* pandas (for data manipulation) 
    
</div>

<div class="alert alert-block alert-danger">
    
## Table of Contents

</div>    

[1. Introduction](#Intro) <br>
[2. Importing Libraries](#libs) <br>
[3. Examining Patent Files](#examine) <br>
[4. Loading and Parsing Files](#load) <br>
$\;\;\;\;$[4.1. Defining Regular Expressions](#Reg_Exp) <br>
$\;\;\;\;$[4.2. Reading Files](#Read) <br>
$\;\;\;\;$[4.3. Whatever else](#latin) <br>
[5. Writing to CSV/JSON File](#write) <br>
$\;\;\;\;$[5.1. Verification - using the sample files](#test_xml) <br>
[6. Summary](#summary) <br>
[7. References](#Ref) <br>

-------------------------------------

<div class="alert alert-block alert-warning">

## 1.  Introduction  <a class="anchor" name="Intro"></a>
    
</div>

This assessment regards extracting data from semi-sctuctured text files. The dataset contained 500 `.txt` files which included various information about user reviews. In particular, ....

-------------------------------------

<div class="alert alert-block alert-warning">
    
## 2.  Importing Libraries  <a class="anchor" name="libs"></a>
 </div>

The packages to be used in this assessment are imported in the following. They are used to fulfill the following tasks:

* **re:** to define and use regular expressions
* **pandas:** ...
* ...

In [2]:
import re
import pandas
import pandas as pd
from datetime import datetime
# My assignment is running locally, and I have already downloaded the data to my local system, so I don't need to link to Google Drive.
# from google.colab import drive
# drive.mount('/content/drive')

-------------------------------------

<div class="alert alert-block alert-warning">

## 3.  Examining Raw Data <a class="anchor" name="examine"></a>

 </div>

First of all, open all the files and examine the data within them.

Having examined the file content, the following observations were made:

Examining the XLSX File:
- By opening the XLSX file, I discovered that there are sheets numbered from 0 to 14. This means I need to combine these sheets into a single DataFrame.
- By observing the different sheets, I noticed that some of them contain entire rows and columns of empty values. I think I need to remove these empty values.
- After removing the irrelevant empty rows and columns, I found that all sheets have the same content. However, the format of the date and other data does not meet the requirements of the assignment.

Examining the TXT Files:
- I have 15 files numbered from 0 to 14. The first line of each TXT file contains the label `<?xml version="1.0" encoding="UTF-8"?>`, which is a common point. I can use this to split the data.
- I also noticed that the label `</dataset><record>` can be used to split the data for each user, and this pattern is consistent across all TXT files.
- In the TXT files, each piece of data is wrapped in tags, such as userid, username, rate, pictures, etc. Additionally, the tags are quite varied; for instance, the rate data can have multiple tag forms such as `<Rate>`, `</rate>`, `<rating>`, `</Rating>`, and more. This means that if I want to extract the data, my regular expressions will need to be compatible with these different tag forms.


-------------------------------------

<div class="alert alert-block alert-warning"> 

## 4.  Loading and Parsing Files <a class="anchor" name="load"></a>

</div>

In this section, the files are parsed and processed. First of all, appropriate regular expressions are defined to extract desired information when reading the files. ....

-------------------------------------

<div class="alert alert-block alert-info">
    
### 4.1. Defining Regular Expressions <a class="anchor" name="Reg_Exp"></a>

Defining correct regular expressions is crucial in extracting desired information from the text efficiently. ...

In [3]:
user_id_expression = r"<[^>]*?ser[^>]*?>\s*(\d{21})\s*<[^>]*?/[^>]*?ser[^>]*?>"
time_expression = r"(?:<.{1,3}ime\s*/?>\s*|<.{0,2}[Dd]ate\s*/?>\s*)(\d+)(?:\s*<\s*/\s*.{1,3}ime\s*/?>|<\s*/\s*.{0,2}[Dd]ate\s*/?>)"
rate_expression = r"(?:<.{0,2}[Rr]ate>\s*)(\d+)(?:<.{0,2}[Rr]ate>)"
rateing_expression = r"(?:<.{0,2}[Rr]ating>\s*)(\d+)(?:<.{0,2}[Rr]ating>)"
text_expression = r"(?:<.{0,3}[Tt]ext>\s*)(.*?)(?:<.{0,3}[Tt]ext>)"
review_expression = r"(?:<.{0,2}[Rr]eview>\s*)(.*?)(?:<.{0,2}[Rr]eview>)"
name_expression = r"(?:<.{0,7}[Nn]ame>\s*)(.*?)(?:<.{0,7}[Nn]ame\s*>)"
response_expression = r"(?:<.{0,3}[Rr]esp.{0,4}>\s*)(.*?)(?:<.{0,3}[Rr]esp.{0,4}>)"
picture_expression = r"(?:<.{0,2}[Pp]ic.{1,5}>\s*)(.*?)(?:<.{0,2}[Pp]ic.{1,5}>)"
gmap_id_expression=r"(?:<.{0,2}[Gg]map.{1,3}>\s*)(.*?)(?:<.{0,2}[Gg]map.{1,3}>)"

In [4]:
text_path = './student_group181'
all_text_data = ""
file_names = [
    'group181_0.txt', 'group181_1.txt', 'group181_2.txt', 'group181_3.txt', 
    'group181_4.txt', 'group181_5.txt', 'group181_6.txt', 'group181_7.txt', 
    'group181_8.txt', 'group181_9.txt', 'group181_10.txt', 'group181_11.txt',
    'group181_12.txt', 'group181_13.txt', 'group181_14.txt'
]
for file_name in file_names:
    file_path = f"{text_path}/{file_name}"
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
        all_text_data += content
        
split_txt = '?xml version="1.0" encoding="UTF-8"?>'
final_split= '</dataset><record>|<dataset><record>'
folder_text = all_text_data.split(split_txt)
if folder_text[0] == "":
    folder_text.pop(0)
final_split_text = []
for text in folder_text:
    slipted_info = re.split(final_split,text)
    final_split_text.extend(slipted_info)

In [5]:
user_id_list = []
gmap_id_list = []
time_list = []
date_list = []
rate_list = []
review_list = []
response_list = []
picture_list = []
name_list = []

user_id_pattern = re.compile(user_id_expression, re.DOTALL)
time_pattern = re.compile(time_expression, re.DOTALL)
rate_pattern = re.compile(rate_expression, re.DOTALL)
rateing_pattern = re.compile(rateing_expression, re.DOTALL)
text_pattern = re.compile(text_expression, re.DOTALL)
review_pattern = re.compile(review_expression, re.DOTALL)
response_pattern = re.compile(response_expression, re.DOTALL)
picture_pattern = re.compile(picture_expression, re.DOTALL)
name_pattern = re.compile(name_expression,re.DOTALL)
gmap_id_pattern = re.compile(gmap_id_expression, re.DOTALL)

for text in final_split_text:
     
     user_id_matches = user_id_pattern.findall(text)
     user_id_list.extend(user_id_matches)
     
     gmap_id_matches = gmap_id_pattern.findall(text)
     gmap_id_list.extend(gmap_id_matches)
     
     time_matches = time_pattern.findall(text)
     time_list.extend(time_matches)
     
     rate_matches = rate_pattern.findall(text)
     if not rate_matches:
        rate_matches = rateing_pattern.findall(text)
     rate_list.extend(rate_matches)
     
     review_matches = review_pattern.findall(text)
     if not review_matches:
        review_matches = text_pattern.findall(text)
     review_list.extend(review_matches)
     
     response_matches = response_pattern.findall(text)
     response_list.extend(response_matches)
     
     picture_matches = picture_pattern.findall(text)
     picture_list.extend(picture_matches)
     
     
     name_matches = name_pattern.findall(text)
     name_list.extend(name_matches)

text_datafram = pd.DataFrame({
    'user_id': user_id_list,
    'name':name_list,
    'time': time_list,
    'rating': rate_list,
    'text': review_list,
    'pics': picture_list,
    'resp': response_list,
    'gmap_id': gmap_id_list
})

text_datafram = text_datafram[['user_id', 'name', 'time', 'rating', 'text', 'pics', 'resp','gmap_id']]
text_datafram['rating'] = text_datafram['rating'].astype(float)
text_datafram['time'] = pd.to_datetime(text_datafram['time'], unit ='ms', utc = True)
text_datafram['time'] = text_datafram['time'].dt.strftime("%Y-%m-%d %H:%M:%S")

text_datafram = text_datafram.replace('None', None)

if_pic = []
if_response = []
for index, row in text_datafram.iterrows():
    if row['pics'] is None:
        if_pic.append('N')
    else:
        if_pic.append('Y')
    
    if row['resp'] is None:
        if_response.append('N')
    else:
        if_response.append('Y')
text_datafram['if_pic'] = if_pic
text_datafram['if_response'] = if_response

text_datafram.info()


/var/folders/5q/zdvmc7js3c31nxcnqbmtsfkc0000gn/T/ipykernel_35552/820920072.py:66: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  text_datafram['time'] = pd.to_datetime(text_datafram['time'], unit ='ms', utc = True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44697 entries, 0 to 44696
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   user_id      44697 non-null  object 
 1   name         44697 non-null  object 
 2   time         44697 non-null  object 
 3   rating       44697 non-null  float64
 4   text         23719 non-null  object 
 5   pics         867 non-null    object 
 6   resp         4637 non-null   object 
 7   gmap_id      44697 non-null  object 
 8   if_pic       44697 non-null  object 
 9   if_response  44697 non-null  object 
dtypes: float64(1), object(9)
memory usage: 3.4+ MB


In [9]:
xlsx_path = "./student_group181/group181.xlsx"
xlsx = pd.ExcelFile(xlsx_path)
all_sheets = pd.read_excel(xlsx_path, sheet_name=None)
cleaned_sheets = []
for sheet_name, df in all_sheets.items():
    df = df.dropna(how='all')
    df = df.dropna(axis=1, how='all')
    if not df.empty:
        cleaned_sheets.append(df)

xlsx_data = pd.concat(cleaned_sheets, ignore_index=True)
xlsx_data = xlsx_data[['user_id', 'name', 'time', 'rating', 'text', 'pics', 'resp','gmap_id']]
xlsx_data['time'] = pd.to_datetime(xlsx_data['time'], unit ='ms', utc = True)
xlsx_data['time'] = xlsx_data['time'].dt.strftime("%Y-%m-%d %H:%M:%S")
xlsx_data['pics'] = xlsx_data['pics'].where(pd.notna(xlsx_data['pics']), None)
xlsx_data['resp'] = xlsx_data['resp'].where(pd.notna(xlsx_data['resp']), None)

if_pic_xlsx = []
if_response_xlsx = []
for index, row in xlsx_data.iterrows():
    # 如果 'pics' 列为 None，添加 'N'，否则添加 'Y'
    if row['pics'] is None:
        if_pic_xlsx.append('N')
    else:
        if_pic_xlsx.append('Y')
    
    # 如果 'resp' 列为 None，添加 'N'，否则添加 'Y'
    if row['resp'] is None:
        if_response_xlsx.append('N')
    else:
        if_response_xlsx.append('Y')

xlsx_data['if_pic'] = if_pic_xlsx
xlsx_data['if_response'] = if_response_xlsx

xlsx_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1782 entries, 0 to 1781
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   user_id      1782 non-null   object 
 1   name         1782 non-null   object 
 2   time         1782 non-null   object 
 3   rating       1782 non-null   float64
 4   text         1019 non-null   object 
 5   pics         29 non-null     object 
 6   resp         151 non-null    object 
 7   gmap_id      1782 non-null   object 
 8   if_pic       1782 non-null   object 
 9   if_response  1782 non-null   object 
dtypes: float64(1), object(9)
memory usage: 139.3+ KB


These patterns are used in the next step when reading the files.

-------------------------------------

<div class="alert alert-block alert-info">
    
### 4.2. Reading Files <a class="anchor" name="Read"></a>

In this step, all files are read and parsed. 

Let's take a look at the first ten elements of the lists generated. We can see that ids, reviews,etc. are parsed and stored correctly.

-------------------------------------

<div class="alert alert-block alert-info">
    
### 4.3. Whatever else <a class="anchor" name="latin"></a>

the rest of your methodology

-------------------------------------

<div class="alert alert-block alert-warning"> 

## 5.  Writing to JSON File <a class="anchor" name="write"></a>

</div>

....

-------------------------------------

<div class="alert alert-block alert-info">
    
### 5.1. Verification of the Generated JSON File <a class="anchor" name="test_xml"></a>

<div class="alert alert-block alert-warning"> 

## 6. Summary <a class="anchor" name="summary"></a>

</div>

....

-------------------------------------

....

-------------------------------------

<div class="alert alert-block alert-warning"> 

## 7. References <a class="anchor" name="Ref"></a>

</div>



[1]<a class="anchor" name="ref-2"></a> Why do I need to add DOTALL to python regular expression to match new line in raw string, https://stackoverflow.com/questions/22610247, Accessed 30/08/2022.

....


## --------------------------------------------------------------------------------------------------------------------------